# Injecting and Measuring Artificial Stars in Single-Visit Images

Jeff Carlin <br>
Last verified to run Monday, April 26, 2022, with weekly 2022_12.

This tutorial demonstrates a method to inject artificial stars into `calexp` images using the measured point-spread function of the given `calexp` image. It then shows how to run the pipelines' detection and measurement tasks on these images, extract the measurements of the artificial stars, and compare their measured magnitudes to the input (simulated) magnitudes.

#### Prerequisites

This tutorial assumes familiarity with afwDisplay, the Butler, and source detection with the LSST Science Pipelines.
Relevant tutorials can be found in the <a href="https://github.com/rubin-dp0/tutorial-notebooks">rubin-dp0/tutorial-notebooks</a> repository.
(For image display, see NBs 03a, 03b, and 08a; for the Butler, see NB 04; and for source detection, see NB 05).

## Introduction

This tutorial shows a way to use existing tools in the LSST Science Pipelines to inject artificial stars into processed single-visit images ("calexps"), then run the detection and measurement tasks to test the recovery of these stars and their (known) properties. In this particular notebook, the focus is on inserting _stars_ into the images because that is the simplest operation (requiring only a position and a flux/magnitude). However, the tools do exist to insert simulated galaxies (based on parameters describing their light distribution), or to insert images (for example, an image of a simulated galaxy cluster) into processed data. See <a href="https://community.lsst.org/t/new-tasks-for-fake-source-insertion/3722">this post</a> on the Community forum for an introduction to the current fake source injection tasks.

**NOTE**: The task that is used in this notebook to create fake stars will soon be deprecated and removed from the Science Pipelines, and its replacement tasks (discussed in the Community forum post linked above) will also soon be replaced with refactored and improved tasks. Once the new synthetic source code has arrived, this notebook should be updated to use the newer tasks.

<br>


The workflow in this notebook is as follows:

1. Starting with an RA, Dec position, find all single-visit images that overlap that position.
    - (For this tutorisal, we select only a single visit image from each band.)
2. Given positions and input magnitudes for fake stars (in _ugri_ bands), figure out what the X, Y positions and fluxes should be in each image.
3. Inject those stars into the appropriate images with the X, Y positions and fluxes as calculated in the previous step, using the measured PSF for each individual image to create stellar images.
4. Run all processing steps (characterization, calibration, detection, deblending, measurement) on each image that fake stars were inserted into.
5. Compare the measured positions and fluxes/mags to the input synthetic values.

## Set Up

#### Import packages.

In [ ]:
# Import python packages
import time
import os
import warnings
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import IFrame, display, Markdown
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.patches import Rectangle
from astropy.visualization import ZScaleInterval
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table

%matplotlib inline

In [ ]:
# Import tasks from the LSST Science Pipelines for reprocessing images
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.pipe.tasks.calibrate import CalibrateTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask

In [ ]:
# Import LSST Science Pipelines packages (see pipelines.lsst.io)
import lsst.daf.base as dafBase
from lsst.daf.butler import Butler
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import lsst.pipe.tasks.fakes as fakes

# Use lsst.afw.display with the matplotlib backend
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [9, 9],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

#### Instantiate the Butler.

In [ ]:
# For DC2 gen3, these are the only optoins
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'

butler = Butler(repo, collections=collection)
registry = butler.registry

#### Create a list of artificial stars to inject: 

To inject artificial stars, all that is needed is a list of RA, Dec positions, and magnitudes to assign the simulated stars.

We will generate `nfakes` stars with positions randomly distributed (using a normal distribution of size `scatter_arcmin` in arcminutes) about the center coordinate (racen, deccen). The stars will be evenly spaced in u-band magnitude (i.e., in `nfakes` equal intervals between magmin and magmax), and randomly distributed about a mean color for each band. (I chose to use a fixed mean color for all stars for simplicity, but apply a normally-distributed scatter of `mag_scatter` magnitudes about this mean value so they are not all identical.)


In [ ]:
# Central (RA, Dec) position (in degrees) at which to insert stars:
racen = 70.376995
deccen = -37.175736

# Place all the fake stars within `scatter_arcmin` arcminutes of the center coordinate:
scatter_arcmin = 1.0

# "nfakes" is the number of fake stars to inject:
nfakes = 31

# Assign positions using a (random) normal distribution about the center coordinates:
fake_ras = racen + (np.random.randn(nfakes) * scatter_arcmin / 60.0)
fake_decs = deccen + (np.random.randn(nfakes) * scatter_arcmin / 60.0)

# Make artificial u magnitudes in a range:
min_umag = 18.0
max_umag = 22.0
mag_scatter = 0.3
fake_umags = np.linspace(min_umag, max_umag, nfakes)

# Now assign magnitudes in other bands using (~fixed) colors:
fake_gmags = fake_umags - (1.0 + mag_scatter * np.random.randn(nfakes))  # distrib centered on u-g=1.0
fake_rmags = fake_gmags - (0.4 + mag_scatter * np.random.randn(nfakes))  # distrib centered on g-r=0.4
fake_imags = fake_rmags - (0.2 + mag_scatter * np.random.randn(nfakes))  # distrib centered on r-i=0.2

# These are totally arbitrary colors, but should be kinda similar to BHB stars?

In [ ]:
# print(fake_ras, fake_decs, fake_gmags)

### Look up the images that overlap the desired coordinates

We will use the spatial query method introduced in [notebook 04: Intro to the Butler](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/04_Intro_to_Butler.ipynb).

In [ ]:
import lsst.sphgeom

pixelization = lsst.sphgeom.HtmPixelization(12)

In [ ]:
htm_id = pixelization.index(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(racen, deccen)
    )
)

# Obtain and print the scale to provide a sense of the size of the sky pixelization being used
scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees()*3600
print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

#### Now query the registry for all "calexp" images overlapping the desired position:

In [ ]:
datasetRefs = registry.queryDatasets("calexp", htm20=htm_id)

print("Found ", datasetRefs.count(), " calexps at that position.")

That's nice, but it finds images from all bands overlapping the position in a single iterator. Let's get lists separately for each band:

In [ ]:
datasetRefs_u = registry.queryDatasets("calexp", htm20=htm_id, band="u")
datasetRefs_g = registry.queryDatasets("calexp", htm20=htm_id, band="g")
datasetRefs_r = registry.queryDatasets("calexp", htm20=htm_id, band="r")
datasetRefs_i = registry.queryDatasets("calexp", htm20=htm_id, band="i")

print("Found ", datasetRefs_u.count(), " u-band calexps at that position.")
print("Found ", datasetRefs_g.count(), " g-band calexps at that position.")
print("Found ", datasetRefs_r.count(), " r-band calexps at that position.")
print("Found ", datasetRefs_i.count(), " i-band calexps at that position.")

For now, let's just grab the first image in the list for each band.

In [ ]:
for i, ref in enumerate(datasetRefs_u):
    uref = ref
    if i > 0:
        break

for i, ref in enumerate(datasetRefs_g):
    gref = ref
    if i > 0:
        break

for i, ref in enumerate(datasetRefs_r):
    rref = ref
    if i > 0:
        break

for i, ref in enumerate(datasetRefs_i):
    iref = ref
    if i > 0:
        break


What does that `datasetRef` object look like?

In [ ]:
iref

### Make stars and inject them into images:

First, write a function ("makeFakeStar") that takes as inputs the position, flux (intensity), and the PSF, and returns an image of the PSF scaled to the desired flux at the input position.

Next, we add a function ("addFakeStars") that takes the list of RAs, Decs, and magnitudes, and a `datasetRef` for a `calexp` image, and returns a version of the input `calexp` that has the fake stars injected into it. (This function calls "makeFakeStar" for each star in the list.)

The source injection steps were developed following this piece of code as an example: https://github.com/lsst/pipe_tasks/blob/387f8f07a2b66205f9fa6bda9a89dcdbbef3f64c/tests/test_fakeProcessing.py

In [ ]:
# make stars at a given position with a given intensity
def makeFakeStar(position, intensity, psf):
    psfImage = psf.computeImage(geom.Point2D(position.x, position.y)).getArray()
    psfImage *= intensity
    noise = np.random.normal(0, np.sqrt(abs(psfImage)))
    return psfImage + noise, noise

In [ ]:
def addFakeStars(ras, decs, mags, calexp_ref):
    calexp_inp = butler.get('calexp', calexp_ref.dataId)
    calexp_withFakes = calexp_inp.clone()

    # Fetch objects from the exposure
    psf = calexp_withFakes.getPsf()
    image = calexp_withFakes.getMaskedImage().getImage()
    mask = calexp_withFakes.getMaskedImage().getMask()
    variance = calexp_withFakes.getMaskedImage().getVariance()
    wcs = calexp_withFakes.getWcs()
    photocalib = calexp_withFakes.getPhotoCalib()

    fluxes = []

    # Use the "photocalib" from the calexp to convert magnitudes to instrumental fluxes:
    for mag in mags:
        fluxes.append(photocalib.magnitudeToInstFlux(mag))

    # Get the corner coordinates of the image:
    y0 = image.getY0()
    x0 = image.getX0()
    ymax = y0+calexp_withFakes.getDimensions()[0]
    xmax = x0+calexp_withFakes.getDimensions()[1]
    
    # Bitplane to set corresponding to the FAKE bit
    # fakeMaskValue = 2**mask.getMaskPlaneDict()['FAKE']
    fakeMaskValue = 2**12
    
    #xvals = []
    #yvals = []
    
    # At each position create a star with the given intensity and add it
    # to the image.
    for ra, dec, intensity in zip(ras, decs, fluxes):
        # Use the WCS to convert RA, Dec to X, Y pixel coordinates:
        pixcoord = wcs.skyToPixel(geom.SpherePoint(ra, dec, geom.degrees))
        pos = positionTuple(pixcoord.y, pixcoord.x)
        #xvals.append(pos.x)
        #yvals.append(pos.y)

        # Call "makeFakeStar" to get the image of a PSF-like object with the desired flux:
        objArray, noiseArray = makeFakeStar(pos, intensity, psf)
        psfRad = int((objArray.shape[0]-1)/2.)

        # Check that the desired position is within the image:
        oky = (pos.y > psfRad/2) & (pos.y < ymax - psfRad/2)
        okx = (pos.x > psfRad/2) & (pos.x < xmax - psfRad/2)

        if okx & oky:
            yslice = slice(int(np.floor(pos.y - psfRad - y0)), int(np.floor(pos.y + psfRad + y0 + 1)))
            xslice = slice(int(np.floor(pos.x - psfRad - x0)), int(np.floor(pos.x + psfRad + x0 + 1)))

            print('Injecting source at y, x = ', pos.y, pos.x)

            image.getArray()[yslice, xslice] += objArray
            mask.getArray()[yslice, xslice] += fakeMaskValue
            variance.getArray()[yslice, xslice] += noiseArray**2
        else:
            print('Skipped injecting source at y, x = ', pos.y, pos.x, ' because it is too close to (or off) the edge.')

    return calexp_inp, calexp_withFakes

### Call the "addFakeStars" function to create calexp images with the artificial stars injected:

In [ ]:
calexp_orig_u, calexp_new_u = addFakeStars(fake_ras, fake_decs, fake_umags, uref)
calexp_orig_g, calexp_new_g = addFakeStars(fake_ras, fake_decs, fake_gmags, gref)
calexp_orig_r, calexp_new_r = addFakeStars(fake_ras, fake_decs, fake_rmags, rref)
calexp_orig_i, calexp_new_i = addFakeStars(fake_ras, fake_decs, fake_imags, iref)

### Wrap all of the detection and measurement steps in a function.

The steps here follow the <a href="https://github.com/rubin-dp0/tutorial-notebooks/blob/main/05_Intro_to_Source_Detection.ipynb">Intro to Source Detection</a> tutorial notebook, so rather than explain them in detail here, we simply wrap all the steps in a single function that can be executed for each image.

In [ ]:
def processFakes(calexp_inp):
    
    # Create a basic schema to use with these tasks
    schema = afwTable.SourceTable.makeMinimalSchema()

    # Create a container which will be used to record metadata about algorithm execution
    algMetadata = dafBase.PropertyList()

    # Initialize CharacterizeImageTask and its configuration:
    config = CharacterizeImageTask.ConfigClass()
    config.psfIterations = 1
    charImageTask = CharacterizeImageTask(None, config=config)

    # Initialize SourceDetectionTask and its configuration:
    config = SourceDetectionTask.ConfigClass()
    config.thresholdValue = 10      # detection threshold in units of thresholdType
    config.thresholdType = "stdev"   # units for thresholdValue
    sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)
    sourceDeblendTask = SourceDeblendTask(schema=schema)

    # Initialize SingleFrameMeasurementTask and its configuration:
    config = SingleFrameMeasurementTask.ConfigClass()
    sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config, algMetadata=algMetadata)

    # Make an empty table that we'll put the results in:
    tab = afwTable.SourceTable.make(schema)
    
    # Image characterization
    result = charImageTask.run(calexp_inp)

    # Get the PSF and some image properties
    psf = calexp_inp.getPsf()
    sigma = psf.computeShape().getDeterminantRadius()
    pixelScale = calexp_inp.getWcs().getPixelScale().asArcseconds()

    # The factor of 2.355 converts from std to fwhm
    print('psf fwhm = {:.2f} arcsec'.format(sigma*pixelScale*2.355))
    
    # Run source detection
    result = sourceDetectionTask.run(tab, calexp_inp)
    
    sources = result.sources
    
    # Run source deblending
    sourceDeblendTask.run(calexp_inp, sources)

    # Run source measurement
    sourceMeasurementTask.run(measCat=sources, exposure=calexp_inp)

    # The copy makes sure that the sources are sequential in memory
    sources = sources.copy(True)
    
    # For convenience, return an Astropy table:
    source_tab = sources.asAstropy()
    
    # Convert fluxes to magnitudes and add them to the table
    photocalib = calexp_inp.getPhotoCalib()
    psfmags = photocalib.instFluxToMagnitude(sources, 'base_PsfFlux')
    source_tab.add_columns([psfmags[:,0], psfmags[:,1]], names=['mag_psf', 'magerr_psf'])
    
    return(source_tab)

### Run the "processFakes" function on each image that we injected fake stars into

**This may take at least a couple minutes to run**

In [ ]:
t1 = time.time()

sources_withfakes_u = processFakes(calexp_new_u)
sources_withfakes_g = processFakes(calexp_new_g)
sources_withfakes_r = processFakes(calexp_new_r)
sources_withfakes_i = processFakes(calexp_new_i)

t2 = time.time()
print('dt', t2 - t1)

In [ ]:
# Take a look at the catalog:

sources_withfakes_i

### Match coordinates from the original list of fake stars to the new catalog:

In [ ]:
# Coordinates of input fake stars
sc_fakes = SkyCoord(ra=fake_ras*u.deg, dec=fake_decs*u.deg, frame='icrs')

# Coordinates of objects in calexps with fakes added:
sc_withfakes_u = SkyCoord(ra=sources_withfakes_u['coord_ra'], dec=sources_withfakes_u['coord_dec'], frame='icrs')
sc_withfakes_g = SkyCoord(ra=sources_withfakes_g['coord_ra'], dec=sources_withfakes_g['coord_dec'], frame='icrs')
sc_withfakes_r = SkyCoord(ra=sources_withfakes_r['coord_ra'], dec=sources_withfakes_r['coord_dec'], frame='icrs')
sc_withfakes_i = SkyCoord(ra=sources_withfakes_i['coord_ra'], dec=sources_withfakes_i['coord_dec'], frame='icrs')
# Note that units weren't required for sc_withfakes_*, because it already has them.

In [ ]:
# Match them using the Astropy SkyCoord "match_to_catalog_sky" method:

idx_u, sep_u, _ = sc_fakes.match_to_catalog_sky(sc_withfakes_u)
idx_g, sep_g, _ = sc_fakes.match_to_catalog_sky(sc_withfakes_g)
idx_r, sep_r, _ = sc_fakes.match_to_catalog_sky(sc_withfakes_r)
idx_i, sep_i, _ = sc_fakes.match_to_catalog_sky(sc_withfakes_i)

The "idx*" arrays are the indices into sc_withfakes* files to extract the closest match, and "sep*" are the actual separations between the sources. Note that this returns a separation for all objects -- in the following, we will keep only sources with match separations less than 1 arcsecond (i.e., "good" matches).

Extract info about the good matches into arrays:

In [ ]:
okmch_u = (sep_u.arcsec < 1.0)
mch_mags_u = sources_withfakes_u[idx_u]['mag_psf']
mch_magerrs_u = sources_withfakes_u[idx_u]['magerr_psf']
# If the nearest match is >1" away, set its magnitudes to -99.9:
mch_mags_u[~okmch_u] = -99.9
mch_magerrs_u[~okmch_u] = 99.9
seps_u = sep_u.arcsec

okmch_g = (sep_g.arcsec < 1.0)
mch_mags_g = sources_withfakes_g[idx_g]['mag_psf']
mch_magerrs_g = sources_withfakes_g[idx_g]['magerr_psf']
mch_mags_g[~okmch_g] = -99.9
mch_magerrs_g[~okmch_g] = 99.9
seps_g = sep_g.arcsec

okmch_r = (sep_r.arcsec < 1.0)
mch_mags_r = sources_withfakes_r[idx_r]['mag_psf']
mch_magerrs_r = sources_withfakes_r[idx_r]['magerr_psf']
mch_mags_r[~okmch_r] = -99.9
mch_magerrs_r[~okmch_r] = 99.9
seps_r = sep_r.arcsec

okmch_i = (sep_i.arcsec < 1.0)
mch_mags_i = sources_withfakes_i[idx_i]['mag_psf']
mch_magerrs_i = sources_withfakes_i[idx_i]['magerr_psf']
mch_mags_i[~okmch_i] = -99.9
mch_magerrs_i[~okmch_i] = 99.9
seps_i = sep_i.arcsec

#print(fake_imags[okmch], sources_withfakes_i[idx_i[okmch]]['mag_psf'])

In [ ]:
# sep_i.arcsec

Combine all of the information into a single table:

In [ ]:
tab_all = Table([fake_ras, fake_decs, fake_umags, fake_gmags, fake_rmags, fake_imags,
                 mch_mags_u, mch_magerrs_u, seps_u, mch_mags_g, mch_magerrs_g, seps_g,
                 mch_mags_r, mch_magerrs_r, seps_r, mch_mags_i, mch_magerrs_i, seps_i],
                 names=['ra', 'dec', 'u_fake', 'g_fake', 'r_fake', 'i_fake', 'u_meas', 'uerr_meas', 'sep_u',
                        'g_meas', 'gerr_meas', 'sep_g', 'r_meas', 'rerr_meas', 'sep_r',
                        'i_meas', 'ierr_meas', 'sep_i'],
                 units=[u.deg, u.deg, u.mag, u.mag, u.mag, u.mag, u.mag, u.mag, u.arcsec,
                        u.mag, u.mag, u.arcsec, u.mag, u.mag, u.arcsec, u.mag, u.mag, u.arcsec])

for col in tab_all.colnames[0:2]:
    tab_all[col].info.format = '%.8g'  # for consistent table output
for col in tab_all.colnames[2:]:
    tab_all[col].info.format = '%.5g'  # for consistent table output


In [ ]:
tab_all

Now we have completed all the steps of injecting fake sources, running pipeline tasks to detect and measure all sources in the resulting images, and extracting the matches from the catalogs. Let's look at the results!

Compare the "fake" magnitudes we assigned to each star to their output (measured) magnitude:

In [ ]:
fig = plt.plot(figsize=(9, 9))

plt.hlines(0, 0, 30, linestyle=':', color='Black')
plt.plot(tab_all['u_fake'], 1000.0 * (tab_all['u_meas'] - tab_all['u_fake']),
         'o', ms=10, color='royalblue', label='u')
plt.plot(tab_all['g_fake'], 1000.0 * (tab_all['g_meas'] - tab_all['g_fake']),
         'D', ms=10, color='mediumseagreen', label='g')
plt.plot(tab_all['r_fake'], 1000.0 * (tab_all['r_meas'] - tab_all['r_fake']),
         '*', ms=10, color='firebrick', label='r')
plt.plot(tab_all['i_fake'], 1000.0 * (tab_all['i_meas'] - tab_all['i_fake']),
         's', ms=10, color='indigo', label='i')
plt.legend()
plt.xlabel(r'$mag_{\rm fake}$')
plt.ylabel(r'$(mag_{\rm meas} - mag_{\rm fake})$ (mmag)')
plt.xlim(15.2, 23.3)
plt.ylim(-48, 48)
plt.minorticks_on()
plt.show()

Looks good! (Note that the y-axis is showing residuals in _millimags_.)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 7))
plt.sca(ax[0])  # set the first axis as current
display1 = afwDisplay.Display(frame=fig)
display1.scale('linear', 'zscale')
display1.mtv(calexp_orig_r.image)
plt.title('original image')
plt.xlim(1000, 2500)
plt.ylim(300, 1800)
plt.sca(ax[1])  # set the second axis as current
display2 = afwDisplay.Display(frame=fig)
display2.scale('linear', 'zscale')
display2.mtv(calexp_new_r.image)
# display2.mtv(image0)
plt.title('with fake stars')
plt.xlim(1000, 2500)
plt.ylim(300, 1800)
plt.tight_layout()
plt.show()

Ideas for further exploration:

- Inject stars using a realistic stellar population (i.e., get colors and a luminosity function from isochrones).
- Inject galaxies or other objects.
- Inject periodic variables into a set of overlapping visit images (using their time of observation to determine the input phase, and thus magnitude, for each image).
- Test completeness of detection/measurement (will need to inject stars over a variety of magnitudes and colors, and probably many times over to build up a large enough statistical sample).
